[Link](https://www.youtube.com/watch?v=KdoGekqz2hg)

In [1]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from binance import Client
import pandas as pd

#Config
import sys
sys.path.insert(0,"/Users/marvinottersberg/Documents/GitHub/sentiment/streamlit")
from config import ConfigBinance
conf = ConfigBinance().getKeys()

client = Client(conf[0],conf[1])

plt.style.use("ggplot")
print(plt.style.available)

asset = "BTCUSDT"



['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


In [2]:
def getminutedata(symbol,interval, lookback):
    klines = client.get_historical_klines(asset,client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")
    print(klines)
    frame = pd.DataFrame(klines)
    #frame = pd.DataFrame(client.get_historical_klines(symbol,interval,lookback))
    frame = frame.iloc[:,:6]
    frame.columns= ["Time","Open","High","Low","Close","Volume"]
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index,unit = "ms")
    frame = frame.astype(float)
    return frame


In [ ]:
def getDateData(symbol,interval, start_str, end_str):
    frame = pd.DataFrame(client.get_historical_klines(symbol,interval,start_str,end_str))
    frame = frame.iloc[:,:6]
    frame.columns= ["Time","Open","High","Low","Close","Volume"]
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index,unit="ms",utc=True)
    #frame.index = frame.index.strftime( "%d-%m-%Y  %H:%M")  
    frame = frame.astype(float)
    return frame

#df = getDateData(asset,"1m","12 July, 2022 20:00:00","12 July,2022 22:00:00")

In [ ]:
df = getminutedata(asset, "1m","120m") #data every 1 minute for total lookback of 120 minutes


In [8]:
def animate(i):
    data = getminutedata(asset,"1m","120m")
    #plt.figure(figsize=((10,8)))
    plt.cla()
    plt.plot(data.index,data.Close)
    plt.xlabel("Time")
    plt.ylabel("Price")
    plt.title(asset)
    plt.gcf().autofmt_xdate() #adjust dates 
    plt.tight_layout()
    

In [9]:
def getPlot():
    ani = FuncAnimation(plt.gcf(),animate,1000) #updating every second
    plt.figure(figsize=((10,8)))
    plt.tight_layout()
    plt.show()
    return ani

anim = getPlot()

<Figure size 432x288 with 0 Axes>

<Figure size 720x576 with 0 Axes>

/Users/marvinottersberg/opt/anaconda3/envs/dontbesentimental/lib/python3.9/site-packages/matplotlib/animation.py:889: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you have outputted the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(


## Read Sentiment from Json and Visualise

In [6]:
import os
import pandas as pd

In [7]:

def getSentiment():
    json_path = "sentiment/Json/12-07-2022/btc.json"

    df = pd.read_json(json_path,orient="index")
    df = df.filter(items=["Time","Sentiment Score"])
    #df_unique = df["Time"].unique()
    df["Time"] = pd.to_datetime(df["Time"])
    df_mean= df.resample("1T",on="Time").mean()
    newdf = pd.DataFrame(df_mean)
    #print(df_mean)
    #print(newdf.to_string())



In [ ]:
def getDatePlotWithSentiment():
    data = getDateData(asset,"1m","12 July, 2022 16:00:00","12 July,2022 22:00:00")
    sent = getSentiment()
    plt.figure(figsize=((12,6)))
    #plt.cla()
    ax1 = plt.subplot(121)
    plt.tick_params('x')
    plt.title(f"Price for {asset}")

    plt.plot(data.index,data.Close)
    
    ax2 = plt.subplot(122,label="Sentiment Score2") #,sharex=ax1)
    # make these tick labels invisible
    plt.tick_params('x')   
    plt.title("Sentiment Score")
    plt.plot(sent)#sent.index,sent["Sentiment Score"].values)
    
    plt.gcf().autofmt_xdate()
    plt.tight_layout()
    plt.show()

In [ ]:
    
# For trendline
    x = df.index.to_pydatetime()
    x = [((y - datetime(1970, 1, 1)) / timedelta(seconds=1)) for y in x]
    y = df["Sentiment Score"]
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    #axs[0].plot(df.index, p(x),color="b",linestyle="dashed")
    
# for coloring
    # for i in y.values:
    #     color1 = plt.cm.viridis(i)
    #     c = [float(i)/float(10), 0.0, float(10-i)/float(10)]
    #     axs[0].plot(x,y,color=color1)